In [2]:
import netCDF4 as nc
import pandas as pd
import os

In [18]:
class DataStruct:
    archive_path = None
    aero_data = {}
    aerodist_data = {}
    wrf_data = {}
    nsh_dict = {}
    boxplot_data = {}
    n_times = None
    n_levels = None 
    domain_x_cells = None
    domain_y_cells = None
    historydelta_m = None

    def addScenario(self, scenario_name, slurm_id, **kwargs):
        start = kwargs.get('start_time', '2023-03-20_09:00:00')
        self.aero_data[scenario_name] =  nc.Dataset(f'{self.archive_path}/slurm-{slurm_id}/aerosols_d01_{start}')
        self.aerodist_data[scenario_name] =  nc.Dataset(f'{self.archive_path}/slurm-{slurm_id}/aerosol_dist_d01_{start}')
        self.wrf_data[scenario_name] = nc.Dataset(f'{self.archive_path}/slurm-{slurm_id}/wrfout_d01_{start}')
        self.nsh_dict[scenario_name] = {}

        if (scenario_name == 'uniform-basecase' and not self.n_times):
            self.addSimAttributes()
    
    def addSimAttributes(self):
        try:
            basecase_aerodata = self.aero_data['uniform-basecase']
            self.n_times = basecase_aerodata.dimensions['Time'].size
            self.n_levels = basecase_aerodata.dimensions['bottom_top'].size
            self.domain_x_cells = basecase_aerodata.dimensions['west_east'].size
            self.domain_y_cells = basecase_aerodata.dimensions['south_north'].size

            times = self.wrf_data['uniform-basecase']['Times'][:].data
            timestamps = [''.join(times[i].astype(str)) for i in range(times.shape[0])]
            timestamps_dt = pd.to_datetime(timestamps, format='%Y-%m-%d_%H:%M:%S')
            self.historydelta_m = (timestamps_dt[1] - timestamps_dt[0]).total_seconds()/60.0

        except KeyError:
            print('uniform-basecase not found in aero_data')
    
    def getScenarioList(self):
        return list(self.aero_data.keys())
    
    def getScenarioSH(self, gridsize):
        path = '/data/keeling/a/sf20/b/wrf-partmc-spatial-het/WRFV3/test/em_les/spatial-het'
        filename = f'sh_patterns_xres{gridsize}_yres{gridsize}_exact.csv'
        dataset = pd.read_csv(os.path.join(path, filename), index_col='scenario')
        scenario_list = self.getScenarioList()
        scenario_sh = {}
        for scenario in scenario_list:
            scenario_sh[scenario] = dataset.loc[scenario, 'NSH']
        return scenario_sh

In [19]:
Data = DataStruct()
Data.archive_path = '/data/keeling/a/sf20/a/wrf-partmc-ideal-coagulation'
Data.addScenario(scenario_name='uniform-basecase', slurm_id=1977120)

In [20]:
Data.getScenarioSH(gridsize=100)

{'uniform-basecase': 0.0}